In [5]:
import os
import json
import pandas as pd
import time
from yelpapi import YelpAPI

# Reading your API key from config.json
def load_config():
    with open('config.json') as f:
        return json.load(f)

config = load_config()
api_key = config['yelp_api_key']

# Function for searching Yelp with pagination
def search_yelp(location, search_term, max_results=240): 
    results = []
    offset = 0
    while offset < max_results:
        with YelpAPI(api_key) as yelp_api:  # Context manager for clean handling
            search_results = yelp_api.search_query(term=search_term, location=location, offset=offset)
            results.extend(search_results['businesses']) 
            offset += len(search_results['businesses'])

            # Respect API Limits (Adjust 'time.sleep' duration if needed)
            time.sleep(0.5)  # Pause for 0.5 seconds between requests
    return pd.DataFrame(results) 

# Example Usage: (Aiming to fetch 240 results)
stores_df = search_yelp('Minneapolis, MN', 'grocery store', max_results=240)
stores_df


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,tIvwPs8Ib8SCEwW45gezPQ,lunds-and-byerlys-minneapolis-5,Lunds & Byerlys,https://s3-media2.fl.yelpcdn.com/bphoto/uS65kV...,False,https://www.yelp.com/biz/lunds-and-byerlys-min...,94,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.6,"{'latitude': 44.9743799, 'longitude': -93.28041}",[],$$,"{'address1': '1201 Hennepin Ave', 'address2': ...",+16123795040,(612) 379-5040,1240.036783
1,zDx8FrqQXGqEffNoxg9sgw,fresh-thyme-market-minneapolis,Fresh Thyme Market,https://s3-media1.fl.yelpcdn.com/bphoto/0F7Uwi...,False,https://www.yelp.com/biz/fresh-thyme-market-mi...,56,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",3.2,"{'latitude': 44.97104719756537, 'longitude': -...",[],$$,"{'address1': '24 30th Ave SE', 'address2': '',...",+16518460908,(651) 846-0908,6032.693511
2,4squJq7UXH7DNj3SrdYpDg,cub-st-louis-park-st-louis-park-2,Cub - St. Louis Park,https://s3-media1.fl.yelpcdn.com/bphoto/NHBUKv...,False,https://www.yelp.com/biz/cub-st-louis-park-st-...,28,"[{'alias': 'grocery', 'title': 'Grocery'}]",2.7,"{'latitude': 44.9681702, 'longitude': -93.3487...",[],$$,"{'address1': '5370 W 16th St', 'address2': '',...",+19525461665,(952) 546-1665,4598.105694
3,Kh0Ftq_Rv72Hy6QOcXp2sA,kowalskis-uptown-market-minneapolis-2,Kowalski's Uptown Market,https://s3-media1.fl.yelpcdn.com/bphoto/QHdnRZ...,False,https://www.yelp.com/biz/kowalskis-uptown-mark...,87,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.7,"{'latitude': 44.95767, 'longitude': -93.29609}","[pickup, delivery]",$$$,"{'address1': '2440 Hennepin Ave', 'address2': ...",+16123773448,(612) 377-3448,968.111708
4,ezFLsLno2NIWa5Bcffy6Qw,target-minneapolis-2,Target,https://s3-media3.fl.yelpcdn.com/bphoto/j1gQ5z...,False,https://www.yelp.com/biz/target-minneapolis-2?...,166,"[{'alias': 'deptstores', 'title': 'Department ...",3.3,"{'latitude': 44.975017, 'longitude': -93.274104}",[],$$,"{'address1': '900 Nicollet Mall', 'address2': ...",+16123380085,(612) 338-0085,1656.678831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,ZDvpfQqjbBHKz92MPFqzbw,aldi-roseville,ALDI,https://s3-media1.fl.yelpcdn.com/bphoto/0xVtzJ...,False,https://www.yelp.com/biz/aldi-roseville?adjust...,13,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",4.3,"{'latitude': 45.02266599895009, 'longitude': -...",[],$,"{'address1': '2005 Twin Lakes Pkwy', 'address2...",+18559552534,(855) 955-2534,10422.139617
236,XQVIX_M2ZRJido8r-qkiRw,hy-vee-plymouth-2,Hy-Vee,https://s3-media3.fl.yelpcdn.com/bphoto/5OlULn...,False,https://www.yelp.com/biz/hy-vee-plymouth-2?adj...,22,"[{'alias': 'grocery', 'title': 'Grocery'}]",2.6,"{'latitude': 45.01966789825101, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '16705 County Rd 24', 'address2':...",+17633837060,(763) 383-7060,17016.027174
237,kA0y-Gg2HkDE6tg5R9fMlg,walgreens-edina,Walgreens,https://s3-media1.fl.yelpcdn.com/bphoto/F5eCZN...,False,https://www.yelp.com/biz/walgreens-edina?adjus...,24,"[{'alias': 'drugstores', 'title': 'Drugstores'...",1.7,"{'latitude': 44.8770642, 'longitude': -93.3205...",[],$$,"{'address1': '6975 York Ave S', 'address2': ''...",+19529203561,(952) 920-3561,10125.002147
238,biSbn-ovng88Mty39eIK-g,kowalskis-markets-eden-prairie-eden-prairie,Kowalski's Markets - Eden Prairie,https://s3-media3.fl.yelpcdn.com/bphoto/KhJa3A...,False,https://www.yelp.com/biz/kowalskis-markets-ede...,27,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.4,"{'latitude': 44.8637594772608, 'longitude': -9...","[pickup, delivery]",$$$,"{'address1': '16500 W 78th St', 'address2': ''...",+19529379585,(952) 937-9585,19147.286810


In [8]:
stores_df.to_csv('yelp_grocery_stores.csv', index=False) 

In [6]:
stores_df.describe()

,review_count,rating,distance
count,240.000000,240.000000,240.000000
mean,43.716667,3.726667,10873.497215
std,61.765542,0.741787,7968.269542
min,1.000000,1.600000,521.043785
25%,12.000000,3.300000,4607.763410
50%,24.000000,3.900000,9426.025863
75%,52.000000,4.200000,15767.128124
max,517.000000,5.000000,41670.368445


In [7]:
stores_df['price'].value_counts()

price
$$      135
$        42
$$$      28
$$$$      1
Name: count, dtype: int64